In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

# Import loaders (used by viewer and overlays)

# Import the CGMViewer object, our central controller

# Import overlays on CGM (glucose) time-series

# Import extensions (e.g. hypnogram) that will be attached in separate panel

# Import CGM-based biomarker overlays

# Import Multi-modal biomarker overlays

In [2]:
# Local path to data if JHE client fails for some reason
base_path = "./sample_subject"

## Loading Data from the JupyterHealth Exchange

In [3]:
# Initialize the JupyterHealth Client
# See documentation for more detail: https://jupyterhealth.org/software-documentation/

# Sample study and patient for the purpose of this tutorial
patient_id = 40079
study_id = 30017

# Get basic information about the patient

### Loading CGM Data from JHE

In [4]:
# Using the client to load our CGM data

In [5]:
# You can also load using the local file
cgm_df = load_cgm_data(source="file", base_path=base_path, filename="blood_glucose.json")
cgm_df.head()

NameError: name 'load_cgm_data' is not defined

### Loading Sleep Data

In [ ]:
# Using the client to load our CGM data

In [ ]:
sleep_df = load_sleep_data(source="file", base_path=base_path, filename="sleep_stage_summary.json")
sleep_df.head()

### Loading Food Entry Data

In [ ]:
# Using the client to load our CGM data

In [ ]:
food_df = load_food_entry_data(source="file", base_path=base_path, filename="food_entry.json")
food_df.head()

## Using the CGMSandbox

In [ ]:
### Initialize the CGMSandbox Viewer and draw daily glucose tracings

### Let's try to add some overlay's to the plain CGM tracing! 

### Now, let us extend the plain CGM tracing by adding an hypnogram extension

### Let us use overlay to visualize our CGM-based biomarkers

### We can utilize sleep and diet data to extract other meaninful markers

In [ ]:
### You can create the same Viewer using local files
viewer = CGMViewer(source="file", 
                   base_path=base_path, 
                   filename="blood_glucose.json",
                   gl_range=(0, 250))

viewer.add_overlay(FoodEntryOverlay(source="file", 
                                    base_path=base_path,
                                    filename="food_entry.json"))

viewer.show()